Nela, encontramos diversos tipos de informações que foram separadas em 4 grupos:

Informação demográfica - 3 variáveis
Doenças pré-existentes - 9 variáveis
Resultados do exame de sangue - 36 variáveis
Sinais vitais - 6 variáveis
Sabemos que há urgência na obtenção e manipulação de dados para melhorar a previsão e assim, conseguir preparar o sistema de saúde, evitando colapsos.

Nosso objetivo será prever quais pacientes precisarão ser admitidos na unidade de terapia intensiva e assim, definir qual a necessidade de leitos de UTI do hospital, a partir dos dados clínicos individuais disponíveis.

Quando conseguimos definir a quantidade de leitos necessários em um determinado hospital, conseguimos evitar rupturas, visto que, caso outra pessoa procure ajuda e, eventualmente, precise de cuidados intensivos, o modelo preditivo já conseguirá detectar essa necessidade e, desta forma, a remoção e transferência deste paciente pode ser organizada antecipadamente.

Queremos que você aplique tudo o que aprendeu durante toda sua trajetória no Bootcamp e construa um modelo com as técnicas de Machine Learning que busquem a nossa variável-resposta.

Tenha em mente que este projeto será apresentado, de maneira fictícia, para o gerente responsável pela modelagem de dados do time de Data Science do Hospital Sírio Libanês. Você precisará persuadi-lo de que seu modelo tem os pontos necessários para entrar em produção e ajudará a antever e evitar qualquer ruptura.

Como a entrega é obrigatória para certificação, montamos um conjunto de critérios mínimos para avaliação que vocês poderão usar como um guia para montar seu estudo.

Temos dois blocos a serem considerados:

Técnico
Prático
Na seção de critérios mínimos deste projeto, você encontra quais são os aspectos que compõem estes blocos e suas respectivas descrições.

Para que o seu projeto seja avaliado pelo Thiago G. Santos e Átila Iamarino, ao vivo, na live de revisão de projetos, submeta seu notebook ou a URL do seu projeto público no GitHub até dia 24/02 às 23h59.}

# Descrição do Problema

A pandemia do novo coronavirus sobrecarregou o sistema hospitalar global durante meses. Despreparados para a demanda longa e volumosa, solicitações por leitos nas Unidades de Tratamento Intensivo (UTI), equipamentos e profissionais ultrapassaram os recursos disponiveis para praticamente todos os hospitais do país. O primeiro caso de COVID-19 no Brasil foi identificado em 26 de Fevereiro de 2020 e desde então, otimizar alocação de recursos na UTI tem sido uma prioridade.

## Call To Action

Sabemos que existe um espectro amplo de casos de COVID-19. Desde pacientes assintomaticos até pacientes necessitando respiração mecânica. Nesse estudo, vamos tentar entender os principais indicadores que levam um paciente para UTI, assim como desenvolver um modelo que nos permita identificar se um paciente precisará ser direcionado para a UTI ou não facilitar a previsão de demanda de leitos de UTI. 

# O Modelo

Para auxiliar na tomada de decisão UTI

### Nossa Variável resposta

Na nossa base de dados, a coluna ```ICU``` indica se aquela observação foi feita na UTI (1) ou não (0). Entender a sutileza desse indicador é extramente importante para nossos proximos passos. A coluna ```ICU``` não descreve quando houve solicitação de um leito de UTI. Além disso, sabemos que a transferencia de um paciente da emergencia/triagem para um leito de UTI pode demorar significativamente. **Esses dois fatores tornam enfraquecem o vínculo das variáveis. ```WINDOW``` e ```ICU```.**

Baseado nisso, nosso modelo se propões a identificar, na triagem inicial, se o paciente deve ser alocado na UTI ou não. Não vai importar para gente quantas horas demorou para ele ser alocado na UTI. Nossa variável resposta vai identificar se o paciente foi eventualmente para a UTI ou não. 

# Dados Faltando


"A falta de respostas também é uma resposta"


## Como é feito o Monitoramento

## Como classificar dados faltantes 

Existem algumas opções de tratamento para dados com ```NaN```. Nesse estudo vamos usar ```NaN = 0```.

Tentar preencher esse valor vazio com médias ou valores obtidos em outras observações simplesmente ignoraria o fato de que, em produção, nosso algoritmo terá de lidar com a ausencia de certos valores, de acordo com a falta de certos tipos de monitoramento ou exames. Portanto, é conceitualmente inviável utilizarmos dados futuros para realizar nossa previsão, pois nossos usuários finais não terão acesso à eles no dia-a-dia. 

Também existe um problema que existe gerado utilizarmos ```NaN = 0```. Porque o valor zero vai passar a descrever duas situações diferentes. zero vai representar tanto a não mensuração, quanto a mensuração igual à zero, que sáo conceitualmente diferentes. Acredito que isso não vai gerar problemas porque (i)mensurações iguais a zero são pouco representativas e (ii)a falta de mensurações também será capturada nas variáveis de monitoramento feito, que definimos na seção anterior.

#

# Importar dados e bibliotecas

In [1]:
import numpy as np
import pandas as pd

!pip install openpyxl

import openpyxl
from itertools import combinations


import warnings
warnings.filterwarnings('ignore')
import os

import matplotlib.pyplot as plt 
import plotly.graph_objects as go


pd.set_option('display.max_columns', 500)

     |████████████████████████████████| 242 kB 3.0 MB/s eta 0:00:01
  Created wheel for et-xmlfile: filename=et_xmlfile-1.0.1-py3-none-any.whl size=8913 sha256=8b93f1d11b2a565b21286e6a4c93344f40407cde94f19bceb49286a5bf6538cb
  Stored in directory: /root/.cache/pip/wheels/e2/bd/55/048b4fd505716c4c298f42ee02dffd9496bb6d212b266c7f31
Successfully built et-xmlfile
You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.


In [2]:
df = pd.read_excel('/kaggle/input/covid19/Kaggle_Sirio_Libanes_ICU_Prediction.xlsx')

# TRATANDO A BASE DE DADOS

In [ ]:
# Para facilitar leitura vamos ordernar o dataframe de acordo com o identificador de cada paciente e a janela da visita desse paciente.
df.sort_values(['PATIENT_VISIT_IDENTIFIER','WINDOW'],inplace=True)

#ajustar para que todas as colunas binárias sigam  o padrão 0 ou 1.
adapt_bool = df[df.columns[df.nunique() == 1]]
adapt_bool = adapt_bool.fillna(0)
adapt_bool = adapt_bool.abs()
df[df.columns[df.nunique() == 1]] = adapt_bool

# Existe um paciente com registros defeituosos. Dado que isso só ocorre com um dos pacientes, vamos eliminar ele do nosso dataset.
df.drop(df[df['PATIENT_VISIT_IDENTIFIER']==199].index,inplace=True)

Existem diversas colunas com valores repetidos. Vamos criar uma função para identificar conjuntos de colunas repetidas, então vamos agregar cada conjunto de colunas repetidas em uma única coluna.

In [ ]:
class RepeatedSet:
    def __init__(self,d,l):
        self.d = d
        self.l = l

def repeated_columns(dataframe):
    
    """This function takes a DataFrame Object and return a dictionary with 
    all sets of columns that contains the same values. 
    
    The keys of the dictionary are the names of the column used to compare values """
    
    temp = dataframe.copy()
    repeated_sets={}

    for j in dataframe.columns:
        if j in temp.columns:
            repeats =[]
            for i in dataframe.columns: 
                repeats.append(temp[j].equals(dataframe[i]))
            if repeats.count(True) > 1:
                repeated_sets[j] = dataframe.columns[repeats].to_list()[1:]
            else:
                pass
            temp.drop(dataframe.columns[repeats].to_list(),axis = 1,inplace=True)
        else:
            pass
    return RepeatedSet(repeated_sets,[item for sublist in repeated_sets.values() for item in sublist])

In [ ]:
#Drop redundant columns
sets= repeated_columns(df)
df.drop(sets.l,axis=1,inplace=True)

#Create map for redudant sets
column_maps = dict(zip(list(sets.d.keys())[2:],[item[:item.rindex('_')] for item in list(sets.d.keys())[2:]]))
column_maps['ALBUMIN_MEDIAN'] = 'ALBUMIN'
column_maps['ALBUMIN_DIFF'] = 'DIFF_SET'
#Rename columns that represent the redudant sets
df.rename(columns=column_maps,inplace=True)

Para esssa análise, também estamos escolhendo não analisar genero do paciente.

In [ ]:
#I dont want to use gender in this analysis
df.drop('GENDER',axis=1,inplace=True)

Podemos agrupar os dados em 4 categorias diferentes:

1. Demográfico
2. Comorbidades
3. Monitoramento
4. Laboratoriais

Todos as métricas laboratoriais e de monitoramento receberam as seguintes caracteristicas

- Média
- Mediana
- Min
- Max

In [ ]:
lab_feat = ['ALBUMIN',
# 'DIFF_SET',           
 'BE_ARTERIAL',
 'BE_VENOUS',
 'BIC_ARTERIAL',
 'BIC_VENOUS',
 'BILLIRUBIN',
 'BLAST',
 'CALCIUM',
 'CREATININ',
 'FFA',
 'GGT',
 'GLUCOSE',
 'HEMATOCRITE',
 'HEMOGLOBIN',
 'INR',
 'LACTATE',
 'LEUKOCYTES',
 'LINFOCITOS',
 'NEUTROPHILES',
 'P02_ARTERIAL',
 'P02_VENOUS',
 'PC02_ARTERIAL',
 'PC02_VENOUS',
 'PCR',
 'PH_ARTERIAL',
 'PH_VENOUS',
 'PLATELETS',
 'POTASSIUM',
 'SAT02_ARTERIAL',
 'SAT02_VENOUS',
 'SODIUM',
 'TGO',
 'TGP',
 'TTPA',
 'UREA',
 'DIMER']

In [ ]:
monit_feat = [ 'BLOODPRESSURE_DIASTOLIC_MEAN',
 'BLOODPRESSURE_SISTOLIC_MEAN',
 'HEART_RATE_MEAN',
 'RESPIRATORY_RATE_MEAN',
 'TEMPERATURE_MEAN',
 'OXYGEN_SATURATION_MEAN',
 'BLOODPRESSURE_DIASTOLIC_MEDIAN',
 'BLOODPRESSURE_SISTOLIC_MEDIAN',
 'HEART_RATE_MEDIAN',
 'RESPIRATORY_RATE_MEDIAN',
 'TEMPERATURE_MEDIAN',
 'OXYGEN_SATURATION_MEDIAN',
 'BLOODPRESSURE_DIASTOLIC_MIN',
 'BLOODPRESSURE_SISTOLIC_MIN',
 'HEART_RATE_MIN',
 'RESPIRATORY_RATE_MIN',
 'TEMPERATURE_MIN',
 'OXYGEN_SATURATION_MIN',
 'BLOODPRESSURE_DIASTOLIC_MAX',
 'BLOODPRESSURE_SISTOLIC_MAX',
 'HEART_RATE_MAX',
 'RESPIRATORY_RATE_MAX',
 'TEMPERATURE_MAX',
 'OXYGEN_SATURATION_MAX',
 'BLOODPRESSURE_DIASTOLIC_DIFF',
 'BLOODPRESSURE_SISTOLIC_DIFF',
 'HEART_RATE_DIFF',
 'RESPIRATORY_RATE_DIFF',
 'TEMPERATURE_DIFF',
 'OXYGEN_SATURATION_DIFF',
 'BLOODPRESSURE_DIASTOLIC_DIFF_REL',
 'BLOODPRESSURE_SISTOLIC_DIFF_REL',
 'HEART_RATE_DIFF_REL',
 'RESPIRATORY_RATE_DIFF_REL',
 'TEMPERATURE_DIFF_REL',
 'OXYGEN_SATURATION_DIFF_REL']

In [ ]:
demo_feat = ['AGE_ABOVE65',
 'AGE_PERCENTIL']

In [ ]:
como_feat = ['DISEASE GROUPING 1',
 'DISEASE GROUPING 2',
 'DISEASE GROUPING 3',
 'DISEASE GROUPING 4',
 'DISEASE GROUPING 5',
 'DISEASE GROUPING 6',
 'HTN',
 'IMMUNOCOMPROMISED',
 'OTHER']

In [ ]:
y = ['ICU']

In [ ]:
featurespace = [demo_feat,
como_feat,
monit_feat,
lab_feat,
y]

In [ ]:
set(df.columns) - set([item for sublist in featurespace for item in sublist])

In [ ]:
df.groupby("PATIENT_VISIT_IDENTIFIER", as_index = False).agg({"ICU":(list), "WINDOW":list})

# Análises

In [ ]:
aux = abs(df.groupby("PATIENT_VISIT_IDENTIFIER")["ICU"].sum()-5)
aux = aux.value_counts().reset_index()
aux.sort_values(by = "index", inplace = True)

aux_map = {0:"0-2", 1:"2-4", 2:"4-6", 3:"6-12", 4:"Above-12",5:"Never"}

aux['index'] = aux['index'].map(aux_map)
aux.rename(columns = {'index':'WINDOW'},inplace=True)
aux.set_index('WINDOW',inplace= True)

total_icu = aux.ICU.sum()
y = aux.ICU[0:5].cumsum()/total_icu

tot_icu_inpatients = aux.ICU[0:5].sum()
y = aux.ICU[0:5].cumsum()/total_icu
plt.plot(y, marker = ".");

# NOVAS FEATURES

## MISSING VALUES ARE STILL VALUES

In [ ]:
df['MISSING DATA'] = df.isnull().sum(axis=1)

In [ ]:
df[df['WINDOW']=='0-2']['MISSING DATA'].unique()

In [ ]:
dict_miss = {}
for i in sorted(df[df['WINDOW']=='0-2']['MISSING DATA'].unique()):
    tempdf = df[(df['MISSING DATA']==i) & (df['WINDOW']=='0-2')]
    
    dict_miss[i] = tempdf.columns[tempdf.isnull().all()].to_list()

In [ ]:
#Break monit_feature into smaller groups
bloodpressure_monit = [idx for idx in monit_feat if idx.startswith('BLOODPRESSURE')]
heart_monit = [idx for idx in monit_feat if idx.startswith('HEART')]
oxygen_monit = [idx for idx in monit_feat if idx.startswith('OXYGEN')]
respiratory_monit = [idx for idx in monit_feat if idx.startswith('RESPIRATORY')]
temperature_monit = [idx for idx in monit_feat if idx.startswith('TEMPERATURE')]

monit_list = [bloodpressure_monit,heart_monit,oxygen_monit,respiratory_monit,temperature_monit]

In [ ]:
def check_monit(missing_list):
    '''given a list of columns with all NaN rows, this function returns what was not being monitored
    false = no monitoring
    true = monitoring'''
    
    monit_list = [bloodpressure_monit,heart_monit,oxygen_monit,respiratory_monit,temperature_monit]
    
    
    print('______group with {} missing values_______'.format(i))
    if set(bloodpressure_monit).issubset(set(missing_list)) == True:
        print('Bloodpressure is NOT being monitored')
    if set(heart_monit).issubset(set(missing_list)) == True:
        print('Heart Rate is NOT being monitored')
    if set(oxygen_monit).issubset(set(missing_list)) == True:
        print('Oxygen level is NOT being monitored')
    if set(respiratory_monit).issubset(set(missing_list)) == True:
        print('Respirartory Frequency is NOT being monitored')
    if set(temperature_monit).issubset(set(missing_list)) == True:
        print('Body Temperature is NOT being monitored')
for i in dict_miss.keys():
    check_monit(dict_miss[i])

Baseado nisso podemos criar features para cada um dos tipos de monitoramento

# LABORATORY

In [ ]:
dict_miss_lab = {}
for i in sorted(df[df['WINDOW']=='0-2']['MISSING DATA'].unique()):
    tempdf = df[(df['MISSING DATA']==i) & (df['WINDOW']=='0-2')]
    tempdf.drop(monit_feat,axis=1, inplace=True)
    
    dict_miss_lab[i] = tempdf.columns[tempdf.isnull().all()].to_list()
    print(i)
    print(tempdf.columns[tempdf.isnull().all()].to_list())

In [ ]:
set(dict_miss_lab[42]) == set(lab_feat)

Podemos usar exames laboratoriais como outra feature

In [ ]:
#Criando novas features para cada conjunto de indicadores

df['missing_lab_exam'] = df[lab_feat].isnull().apply(lambda x: all(x), axis=1).astype(int)
df['missing_bloodpresure_monit'] = df[bloodpressure_monit].isnull().apply(lambda x: all(x), axis=1).astype(int)
df['missing_heart_monit'] = df[heart_monit].isnull().apply(lambda x: all(x), axis=1).astype(int)
df['missing_oxygen_monit'] = df[oxygen_monit].isnull().apply(lambda x: all(x), axis=1).astype(int)
df['missing_respiratory_monit'] = df[respiratory_monit].isnull().apply(lambda x: all(x), axis=1).astype(int)
df['missing_temperature_monit'] = df[temperature_monit].isnull().apply(lambda x: all(x), axis=1).astype(int)

In [6]:
#Conferindo se existe ocorrencias de NaN que não são explicadas pelos 6 conjuntos acima

(df['MISSING DATA']\
- (len(lab_feat)* df['missing_lab_exam'])\
- (len(bloodpressure_monit)* df['missing_bloodpresure_monit'])\
- (len(heart_monit)* df['missing_heart_monit'])\
- (len(oxygen_monit)* df['missing_oxygen_monit'])\
- (len(respiratory_monit)* df['missing_respiratory_monit'])\
- (len(temperature_monit)* df['missing_temperature_monit'])).unique()

KeyError: 'MISSING DATA'

# Preencher os NaN

## Porque NaN =0

# Mais algo?

# OTHERS

In [ ]:
#Deal with WINDOW and AGE_PERCENTIL variables
dummies =  pd.get_dummies(df['AGE_PERCENTIL'])

df =pd.concat([df,dummies],axis=1)
df.drop('AGE_PERCENTIL',axis=1,inplace=True)

In [ ]:
df

In [ ]:
target_var =  df[['PATIENT_VISIT_IDENTIFIER','ICU']].groupby('PATIENT_VISIT_IDENTIFIER').sum('ICU')

target_var['target']= target_var['ICU']>0
target_var['target']= target_var['target'].astype('int')
target_var.drop('ICU',axis=1,inplace=True)

target_var

In [ ]:
y= target_var['target']

# Model Trials

## Ver. 1
- replace NaNs with zero
- use only 0-2 window

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(model1, y, test_size=0.3,stratify=y, random_state=11111993)

In [ ]:
from sklearn.impute   import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics  import accuracy_score, auc, roc_curve, precision_recall_curve, roc_auc_score, precision_score, recall_score, average_precision_score
from sklearn.model_selection import train_test_split, StratifiedKFold


In [ ]:
def model_evaluation(model, testing_set_x, testing_set_y):
    predictions = model.predict_proba(testing_set_x)
    
    accuracy  = accuracy_score(testing_set_y, predictions[:,1] >= 0.5)
    roc_auc   = roc_auc_score(testing_set_y, predictions[:,1])
    precision = precision_score(testing_set_y, predictions[:,1] >= 0.5)
    recall    = recall_score(testing_set_y, predictions[:,1] >= 0.5)
    pr_auc    = average_precision_score(testing_set_y, predictions[:,1])
    
    result = pd.DataFrame([[accuracy, precision, recall, roc_auc, pr_auc]], columns=['Accuracy', 'Precision', 'Recall', 'ROC_auc','PR_auc'])
    return(result)

In [ ]:
rf_hyperparameters = {
              'n_estimators':2100,
              'max_depth':27,
              'min_samples_split':2,
              'min_samples_leaf':4,
              'random_state':451,
            }


clf = RandomForestClassifier(**rf_optimal)

clf.fit(X_train,y_train)

model_evaluation(clf, X_test,y_test)

In [ ]:
model1 = df.copy()
model1 = model1[model1['WINDOW'] == '0-2']
model1 = model1.fillna(0)
model1 = model1.reset_index()

model1.drop(['index','PATIENT_VISIT_IDENTIFIER','MISSING DATA','WINDOW'],axis=1,inplace=True)


# Random Forest

In [ ]:
model2 = df.copy()
model2 = model2.fillna(0)

target_mapping = target_var['target'].to_dict()
model2['target'] = model2['PATIENT_VISIT_IDENTIFIER'].map(target_mapping)

model2.reset_index(drop=True)
model2.drop(['PATIENT_VISIT_IDENTIFIER','MISSING DATA','WINDOW'],axis=1,inplace=True)

y = model2.pop('target')
X = model2

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3,stratify=y, random_state=11111993)

In [ ]:
rf_hyperparameters = {
              'n_estimators':2100,
              'max_depth':27,
              'min_samples_split':2,
              'min_samples_leaf':4,
              'random_state':451,
            }


clf = RandomForestClassifier(**rf_optimal)

clf.fit(X_train,y_train)

model_evaluation(clf, X_test,y_test)